# Query Ocean Sensitive Areas with your Asset Data
### A script that automates the process of querying the HUB Ocean Ocean Sensitive Area Data Product to return results based on your own asset data.

In [1]:
!pip install -U odp-sdk --quiet

DEPRECATION: Loading egg at /srv/conda/envs/notebook/lib/python3.11/site-packages/odp-0.0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.17.56 requires cryptography<40.0.2,>=40.0.0, but you have cryptography 44.0.2 which is incompatible.
pyopenssl 23.2.0 requires cryptography!=40.0.0,!=40.0.1,<42,>=38.0.0, but you have cryptography 44.0.2 which is incompatible.


In [2]:
pip install h3 --quiet

DEPRECATION: Loading egg at /srv/conda/envs/notebook/lib/python3.11/site-packages/odp-0.0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [1]:
from odp.client import OdpClient # The SDK
from shapely.geometry import box
import pandas as pd
import pydeck as pdk
import os
import h3
from ipywidgets import widgets
from IPython.display import display, clear_output

In [2]:
base_url = "https://odcat.dev.hubocean.io"
client = OdpClient(base_url=base_url)

## Extract lat and long from CSV and add Hex6 collumn
### Expects a csv file called "asset.csv" with required columns: "asset_id", "lat", "long"

In [9]:
def load_and_process_asset_data(file_path: str = "asset.csv", max_size_mb: int = 5, distance_km: int = 50) -> pd.DataFrame:
    """
    Loads asset data from a CSV file, verifies required columns (case insensitive),
    and adds an H3 index (resolution 6) column based on latitude and longitude.
    Also computes surrounding H3 indexes within a specified distance.
    """
    max_size_bytes = max_size_mb * 1024 * 1024  # Convert MB to Bytes
    file_size = os.path.getsize(file_path)
    if file_size > max_size_bytes:
        raise ValueError(f"Error: File size exceeds {max_size_mb}MB limit ({file_size / (1024*1024):.2f}MB).")
    
    try:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path, encoding="utf-8")
        elif file_path.endswith(('.xls', '.xlsx')):
            df = pd.read_excel(file_path)
        else:
            raise ValueError("Unsupported file format. Use CSV or Excel (.xls/.xlsx).")
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding="ISO-8859-1")
        
    df.columns = df.columns.str.lower()
    lat_columns = {'latitude', 'lat'}
    lon_columns = {'longitude', 'long', 'lon'}
    
    lat_col = next((col for col in df.columns if col in lat_columns), None)
    lon_col = next((col for col in df.columns if col in lon_columns), None)
    
    if not lat_col or not lon_col:
        raise ValueError("Error: The file must contain 'latitude' and 'longitude' (or 'lat' and 'long') columns.")
    
    df['h3_index'] = df.apply(lambda row: h3.latlng_to_cell(row[lat_col], row[lon_col], 6), axis=1)
    
    def get_h3_neighbors(h3_index):
        num_rings = round(distance_km / 8.74)  # Approximate conversion for resolution 6
        return list(h3.grid_disk(h3_index, num_rings))
    
    df['h3_neighbors'] = df['h3_index'].apply(get_h3_neighbors)
    return df

def interactive_h3_processing():
    global df  # Ensure df is accessible globally
    file_path = "asset.csv"
    
    distance_slider = widgets.IntSlider(
        value=50,
        min=10,
        max=200,
        step=5,
        description='Distance (km):',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
        style={'description_width': 'initial'}
    )
    
    process_button = widgets.Button(
        description='Process Data',
        button_style='primary',
        tooltip='Click to process the file',
        icon='check'
    )
    
    output = widgets.Output()
    
    def on_button_clicked(b):
        global df  # Ensure df is accessible globally
        with output:
            clear_output()
            try:
                print(f"Processing file: {file_path}")
                print(f"Distance for H3 neighbors: {distance_slider.value} km")
                
                df = load_and_process_asset_data(
                    file_path=file_path,
                    distance_km=distance_slider.value
                )
                
                print("\nProcessed data sample (first 5 rows):")
                display(df.head(5))
                
                print(f"\nTotal records processed: {len(df)}")
                avg_neighbors = df['h3_neighbors'].apply(len).mean()
                print(f"Average number of neighboring hexagons: {avg_neighbors:.2f}")
            
            except Exception as e:
                print(f"Error: {str(e)}")
    
    process_button.on_click(on_button_clicked)
    
    display(widgets.HTML(f"<h3>H3 Geospatial Data Processor</h3><p>File: <b>{file_path}</b></p>"))
    display(distance_slider)
    display(process_button)
    display(output)
    
    return {
        'distance': distance_slider,
        'output': output
    }

## Define your search area around assets

In [10]:
interactive_h3_processing()

HTML(value='<h3>H3 Geospatial Data Processor</h3><p>File: <b>asset.csv</b></p>')

IntSlider(value=50, continuous_update=False, description='Distance (km):', max=200, min=10, step=5, style=Slid…

Button(button_style='primary', description='Process Data', icon='check', style=ButtonStyle(), tooltip='Click t…

Output()

{'distance': IntSlider(value=50, continuous_update=False, description='Distance (km):', max=200, min=10, step=5, style=SliderStyle(description_width='initial')),
 'output': Output()}

## Query Ocean Sensitive Areas Dataset

In [11]:
# Create a mapping of asset_id to h3_index and its neighbors
asset_h3_mapping = []

for _, row in df.iterrows():
    asset_h3_mapping.append({
        'asset_id': row['asset_id'],
        'h3_index': row['h3_index'],
        'h3_neighbors': row['h3_neighbors']
    })

# Print first 5 entries for verification
print("Asset Locations with Neighbors:")
for entry in asset_h3_mapping[:5]:
    print(f"Asset ID: {entry['asset_id']}, H3 Index: {entry['h3_index']}")
    print(f"Neighbors: {entry['h3_neighbors'][:3]}... ({len(entry['h3_neighbors'])} total neighbors)\n")


Asset Locations with Neighbors:
Asset ID: 1, H3 Index: 864ce832fffffff
Neighbors: ['864ce832fffffff', '864ce8327ffffff', '864ce8307ffffff']... (127 total neighbors)

Asset ID: 2, H3 Index: 864cee4efffffff
Neighbors: ['864cee4efffffff', '864cee4e7ffffff', '864cee4c7ffffff']... (127 total neighbors)

Asset ID: 3, H3 Index: 8645a322fffffff
Neighbors: ['8645a322fffffff', '8645a3227ffffff', '8645a3207ffffff']... (127 total neighbors)

Asset ID: 4, H3 Index: 865e4b437ffffff
Neighbors: ['865e4b437ffffff', '865e4b5afffffff', '865e4b58fffffff']... (127 total neighbors)

Asset ID: 5, H3 Index: 864cd48dfffffff
Neighbors: ['864cd48dfffffff', '864cd48d7ffffff', '864cd412fffffff']... (127 total neighbors)



In [12]:
## Request the dataset from the catalog using the UUID:
osa_table_dataset = client.catalog.get(("7af5bc0f-c12a-451b-9834-fa6c4e0a2c87"))
osa_data = client.table_v2(osa_table_dataset)
osa_table_dataset.metadata.display_name

'Ocean Sensitive Areas - version 1.0.0 - table'

In [13]:
# Initialize an empty list to store the results for all assets
all_results = []

# Loop through each asset in the DataFrame
for _, row in df.iterrows():
    # Get the asset's H3 index and neighbors
    hex6_set = {row['h3_index']}  # Add the asset's H3 index
    hex6_set.update(row['h3_neighbors'])  # Add its neighbors

    # Convert the set to a list for query construction
    hex6_array = list(hex6_set)

    # Build the query using OR conditions
    query = " OR ".join(f'hex6 == "{h}"' for h in hex6_array)

    # Execute the query to get results for the current asset and its neighbors
    result_df = next(osa_data.select(query).dataframes())

    # Add a column to indicate whether the record is a neighbor or the main asset
    result_df['is_neighbor'] = result_df['hex6'].apply(
        lambda x: 'Neighbor' if x != row['h3_index'] and x in row['h3_neighbors'] else 'Asset'
    )

    # Add the asset_id to the results (the current asset's ID)
    result_df['asset_id'] = row['asset_id']
    result_df['lat'] = row['lat']
    result_df['long'] = row['long']

    # Append the result to the all_results list
    all_results.append(result_df)

# Concatenate all the individual results into a single DataFrame
final_results_df = pd.concat(all_results, ignore_index=True)

## Results printed as a table and saved as csv

In [14]:
# Function to save the DataFrame as a CSV
def save_to_csv(df, filename="osa_results.csv"):
    try:
        df.to_csv(filename, index=False)
        print(f"Data successfully saved to {filename}")
    except Exception as e:
        print(f"Error saving data: {e}")

# Assuming final_results_df is already created
print(f"Total rows in final result: {len(final_results_df)}")
# print(final_results_df.head(5))

# Save the DataFrame to a CSV file
save_to_csv(final_results_df)


Total rows in final result: 672
Data successfully saved to osa_results.csv


In [15]:
final_results_df.head(5)

,shannon,simpson,mangrove,hex6,seamount,cold_water_coral,seagrass,geometry,coral,S,shannon_norm,S_raw,is_neighbor,asset_id,lat,long
0,3.007,0.063,0.0,864ce800fffffff,0.0,0.0,1.0,"POLYGON ((-64.9679 18.2659, -64.944 18.2872, -...",1.0,40.196,0.376,0.396,Neighbor,1,18.2208,-64.7034
1,4.771,0.019,0.0,864ce802fffffff,0.0,0.0,1.0,"POLYGON ((-64.9482 18.315, -64.9243 18.3364, -...",1.0,43.831,0.596,0.433,Neighbor,1,18.2208,-64.7034
2,4.342,0.018,0.0,864ce8047ffffff,0.0,0.0,1.0,"POLYGON ((-64.8837 18.2467, -64.8598 18.2681, ...",1.0,42.946,0.542,0.424,Neighbor,1,18.2208,-64.7034
3,4.092,0.023,0.0,864ce804fffffff,0.0,0.0,0.0,"POLYGON ((-64.8515 18.2125, -64.8276 18.2339, ...",0.0,9.432,0.511,0.085,Neighbor,1,18.2208,-64.7034
4,4.190,0.021,0.0,864ce8057ffffff,0.0,0.0,0.0,"POLYGON ((-64.9357 18.2317, -64.9118 18.2531, ...",1.0,26.133,0.523,0.254,Neighbor,1,18.2208,-64.7034


In [23]:
def categorize_shannon(shannon):
    if shannon < 2.0:
        return "Low Biodiversity"
    elif 2.0 <= shannon <= 4.0:
        return "Medium Biodiversity"
    else:
        return "High Biodiversity"

def categorize_simpson(simpson):
    if simpson > 0.5:
        return "High Dominance, Low Evenness"
    elif 0.2 <= simpson <= 0.5:
        return "Moderate Dominance"
    else:
        return "Low Dominance, High Evenness"

def generate_asset_report(df):
    global radius_km_global  # Ensure radius_km_global is accessible
    report_list = []
    
    # Compute asset ranking based on average Shannon Index
    asset_ranks = df.groupby("asset_id")["shannon"].mean().rank(ascending=False).to_dict()
    df["biodiversity_rank"] = df["asset_id"].map(asset_ranks)
    
    # Sort assets by biodiversity rank (higher Shannon Index first)
    sorted_assets = df.groupby("asset_id")["shannon"].mean().sort_values(ascending=False).index
    
    for asset_id in sorted_assets:
        asset_df = df[df['asset_id'] == asset_id]
        
        # Extract exact location row
        exact_location = asset_df[asset_df['is_neighbor'].str.lower() == "asset"]
        if not exact_location.empty:
            exact_shannon = exact_location['shannon'].values[0]
            exact_simpson = exact_location['simpson'].values[0]
            ecosystems = [eco.capitalize() for eco in ['mangrove', 'seamount', 'cold_water_coral', 'seagrass', 'coral'] 
                          if exact_location[eco].values[0] > 0]
        else:
            exact_shannon, exact_simpson, ecosystems = None, None, []

        # Extract surrounding data (use global radius_km_global instead of hardcoded 50 km)
        neighbors = asset_df[asset_df['is_neighbor'].str.lower() == "neighbor"]
        avg_shannon = neighbors['shannon'].mean() if not neighbors.empty else None
        avg_simpson = neighbors['simpson'].mean() if not neighbors.empty else None

        # Rank and total assets
        asset_rank = asset_ranks.get(asset_id, None)
        total_assets = df['asset_id'].nunique()
        
        # Helper function to format with default value
        def safe_format(value):
            return f"{value:.3f}" if value is not None else "N/A"
        
        # Construct Report
        report = f"""
Asset ID: {asset_id}
Biodiversity Rank: #{int(asset_rank) if asset_rank else "N/A"} out of {total_assets}
-----------------------------------
Exact Location:
  - Shannon Index: {safe_format(exact_shannon)} ({categorize_shannon(exact_shannon) if exact_shannon is not None else "N/A"})
  - Simpson Index: {safe_format(exact_simpson)} ({categorize_simpson(exact_simpson) if exact_simpson is not None else "N/A"})
  - Ecosystems: {', '.join(ecosystems) if ecosystems else "None"}

Surrounding Area ({radius_km_global}km radius):
  - Avg Shannon Index: {safe_format(avg_shannon)} ({categorize_shannon(avg_shannon) if avg_shannon is not None else "N/A"})
  - Avg Simpson Index: {safe_format(avg_simpson)} ({categorize_simpson(avg_simpson) if avg_simpson is not None else "N/A"})
  - % Coverage: 
    - Coral: {safe_format(neighbors['coral'].mean() * 100 if not neighbors.empty else None)}%
    - Seagrass: {safe_format(neighbors['seagrass'].mean() * 100 if not neighbors.empty else None)}%
    - Cold Water Coral: {safe_format(neighbors['cold_water_coral'].mean() * 100 if not neighbors.empty else None)}%
"""
        report_list.append(report)
    
    return report_list

# Run the function and print reports
reports = generate_asset_report(final_results_df)
for r in reports:
    print(r)



Asset ID: 6
Biodiversity Rank: #1 out of 20
-----------------------------------
Exact Location:
  - Shannon Index: 3.616 (Medium Biodiversity)
  - Simpson Index: 0.033 (Low Dominance, High Evenness)
  - Ecosystems: Seagrass, Coral

Surrounding Area (50km radius):
  - Avg Shannon Index: 2.842 (Medium Biodiversity)
  - Avg Simpson Index: 0.200 (Moderate Dominance)
  - % Coverage: 
    - Coral: 65.432%
    - Seagrass: 55.556%
    - Cold Water Coral: 1.235%


Asset ID: 3
Biodiversity Rank: #2 out of 20
-----------------------------------
Exact Location:
  - Shannon Index: 1.946 (Low Biodiversity)
  - Simpson Index: 0.143 (Low Dominance, High Evenness)
  - Ecosystems: Mangrove, Seagrass, Coral

Surrounding Area (50km radius):
  - Avg Shannon Index: 2.242 (Medium Biodiversity)
  - Avg Simpson Index: 0.248 (Moderate Dominance)
  - % Coverage: 
    - Coral: 80.000%
    - Seagrass: 84.000%
    - Cold Water Coral: 52.000%


Asset ID: 19
Biodiversity Rank: #2 out of 20
--------------------------